# ETL 03 - Scripts Area Stage SQL - Geolocalizacao
    Objetivo: Popular a tabelas TOiSTG_CidadeGeoLocalizacao no banco DSTGOiMasterDados
    Desenvolvedor: Marcelo Muniz de Alencar

In [ ]:
!pip install geopy # instalar geopy

  Using cached geopy-2.2.0-py3-none-any.whl (118 kB)
  Using cached geographiclib-1.52-py3-none-any.whl (38 kB)


In [ ]:
from geopy.geocoders import Nominatim
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
connection = pyodbc.connect("Driver={SQL Server};Server=PCMARCELO\\SQLEXPRESS;Database=DSTG_OiMasterDados;Trusted_Connection=yes;") 
sql = ''' SELECT  NumCidade,
                  case when rtrim(ltrim(Estado)) <> ''
                        then Cidade + ', ' + Estado + ', '+ Pais
                        else Cidade + ', ' + Pais
                  end as Localizacao
          FROM OiMasterDados.TOiSTG_Cidade (NOLOCK) 
          ORDER BY NumCidade '''
df = pd.read_sql(sql,connection)

listaLocalizacao = df['Localizacao'].tolist() # criar lista
listaLocalizacao

['Algiers, Algeria',
 'Cotonou, Benin',
 'Bujumbura, Burundi',
 'Bangui, Central African Republic',
 'Brazzaville, Congo',
 'Cairo, Egypt',
 'Addis Ababa, Ethiopia',
 'Libreville, Gabon',
 'Banjul, Gambia',
 'Conakry, Guinea',
 'Bissau, Guinea-Bissau',
 'Abidjan, Ivory Coast',
 'Nairobi, Kenya',
 'Antananarivo, Madagascar',
 'Lilongwe, Malawi',
 'Nouakchott, Mauritania',
 'Rabat, Morocco',
 'Maputo, Mozambique',
 'Windhoek, Namibia',
 'Lagos, Nigeria',
 'Niamey, Nigeria',
 'Dakar, Senegal',
 'Freetown, Sierra Leone',
 'Capetown, South Africa',
 'Dar Es Salaam, Tanzania',
 'Lome, Togo',
 'Tunis, Tunisia',
 'Kampala, Uganda',
 'Lusaka, Zambia',
 'Dhaka, Bangladesh',
 'Beijing, China',
 'Chengdu, China',
 'Guangzhou, China',
 'Shanghai, China',
 'Shenyang, China',
 'Hong Kong, Hong Kong',
 'Bombay (Mumbai), India',
 'Calcutta, India',
 'Chennai (Madras), India',
 'Delhi, India',
 'Jakarta, Indonesia',
 'Osaka, Japan',
 'Sapporo, Japan',
 'Tokyo, Japan',
 'Almaty, Kazakhstan',
 'Bishkek, K

In [ ]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

cursor = connection.cursor()
cursor.execute('DELETE FROM [DSTG_OiMasterDados].[OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]')
connection.commit()

for i in listaLocalizacao:
    try:
        location = geolocator.geocode(i)
        #print(i) 
        #print(str(location.latitude) + ',' + str(location.longitude))
        sqlInsert =  ''' INSERT INTO [OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]
                               ([Localizacao]
                               ,[ValLatitude]
                               ,[ValLongitude])
                         VALUES
                           ( ''' + "'" + i + "'" + ', ' + str(location.latitude) + ', ' + str(location.longitude) + ')' 
        #print(sqlInsert)
        cursor.execute(sqlInsert)
        connection.commit()
    except:
        print("Erro na cidade " + i)

Erro na cidade Pristina, Serbia-Montenegro
Erro na cidade San Juan Puerto Rico, Additional Territories, US
Erro na cidade Montvideo, Uruguay


In [ ]:
sql2 = ''' SELECT  case when T1.[Cidade] = 'Montvideo' 
                            then 'Montevideo' 
                            else T1.[Cidade]
                    end as Localizacao
            FROM  [DSTG_OiMasterdados].[OiMasterDados].[TOiSTG_Cidade] AS T1 (NOLOCK) 
                        left outer join
                  [DSTG_OiMasterdados].[OiMasterDados].[TOiSTG_CidadeGeoLocalizacao] AS T2 (NOLOCK)
                        on (case when rtrim(ltrim(T1.Estado)) <> ''
                                    then T1.Cidade + ', ' + T1.Estado + ', '+ T1.Pais
                                    else T1.Cidade + ', ' + T1.Pais
                            end) = T2.Localizacao
            WHERE T2.Localizacao is null '''
df2 = pd.read_sql(sql2,connection)

listaLocalizacao2 = df2['Localizacao'].tolist() # criar lista
listaLocalizacao2

['San Juan Puerto Rico', 'Pristina', 'Montevideo']

In [ ]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")

for i in listaLocalizacao2:
    try:
        location2 = geolocator.geocode(i)
        #print(i) 
        #print(str(location2.latitude) + ',' + str(location2.longitude))
        sqlInsert2 =  ''' INSERT INTO [OiMasterDados].[TOiSTG_CidadeGeoLocalizacao]
                               ([Localizacao]
                               ,[ValLatitude]
                               ,[ValLongitude])
                         VALUES
                           ( ''' + "'" + i + "'" + ', ' + str(location2.latitude) + ', ' + str(location2.longitude) + ')' 
        #print(sqlInsert2)
        cursor.execute(sqlInsert2)
        connection.commit()
    except:
        print("Erro na cidade " + i)

## FIM